In [1]:
import re
import csv
import time
import nltk
import json
import string
import numpy as np
import pandas as pd
from nltk import stem
import sklearn.metrics
from random import randint
from numpy.linalg import norm
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [2]:
with open('../new_userDict.json', 'r') as fp:
    
    new_userDict = json.load(fp)
    
    
with open('../new_isbnDict.json', 'r') as fp:
    
    new_isbnDict = json.load(fp)
    
    
with open('../dict_row.json', 'r') as fp:
    
    dict_row = json.load(fp)
    
    
with open('../dict_col.json', 'r') as fp:
    
    dict_col = json.load(fp)
    
print("Ok") 

Ok


In [3]:
with open('../clustering/CLUSTERS_ITEMS.json', 'r') as fp:
    
    CLUSTERS_ITEMS = json.load(fp)
    
    
with open('../clustering/CLUSTERS_USERS.json', 'r') as fp:
    
    CLUSTERS_USERS = json.load(fp)
    
    
with open('../clustering/file/clusters_dict_row.json', 'r') as fp:
    
    clusters_dict_row = json.load(fp)
    
    
with open('../clustering/file/clusters_dict_col.json', 'r') as fp:
    
    clusters_dict_col = json.load(fp)
    
print("Ok") 



with open('../clustering/index_book_user_clusters/index_user_cluster.json', 'r') as fp:
    
    index_user_cluster = json.load(fp)
    
    
with open('../clustering/index_book_user_clusters/index_book_cluster.json', 'r') as fp:
    
    index_book_cluster = json.load(fp)
    
print("Ok")

Ok
Ok


In [4]:
def generateClustersUsingClustersNewDicts(clusters_new_isbnDict, index_user_cluster):

    clusters_books = {k:{} for k in clusters_new_isbnDict}

    for c_i in clusters_new_isbnDict:

        for user in clusters_new_isbnDict[c_i]:

            try:
                c_u = index_user_cluster[user]

                if c_u not in clusters_books[c_i]:

                    clusters_books[c_i][c_u] = []

                if float(clusters_new_isbnDict[c_i][user]) != 0.0:

                    clusters_books[c_i][c_u].append(float(clusters_new_isbnDict[c_i][user]))

            except:
                continue
                
    for c_b in clusters_books:

        users_cluster = list(clusters_books[c_b].keys())

        for c_u in users_cluster:

            if clusters_books[c_b][c_u] == []:

                del clusters_books[c_b][c_u]
            else:
                clusters_books[c_b][c_u] = np.mean(clusters_books[c_b][c_u])
                
                
    return clusters_books

In [5]:
# ATTENTION!!!  different respect to usersHaveRatedBook(s)

def usersHaveRatedBook(new_isbnDict, new_userDict, book_number, score):
    
    
    '''
    
    input:  new_isbnDict (Dict), new_userDict (Dict), book_number (int), score (int)
    
    action: select all users that have given a good rating to book_number
    
    output: users_rated (list)
    
    '''
    
    users_rated = []
    
    for user in new_isbnDict[str(book_number)]:
        
        if float(new_isbnDict[str(book_number)][user]) > score:
            
            try:
                
                new_userDict[user]
                users_rated.append(user)
                
            except:
                
                continue
            
    return list(set(users_rated))


def SimilarityUsers(utility_DataFrame, user_number, users_similar):
    
    '''
    
    input: utility_DataFrame (DataFrame), user_number (int), user_similar (List)
    
    action: compute cosine similarity between user_number and all the user in users_similar
    
    output: new_similarity (List of tuples)
    
    '''

    x = utility_DataFrame.loc[str(user_number)]
    x_length = norm(x)
    
    y = utility_DataFrame.loc[users_similar]
    y_length = norm(utility_DataFrame.loc[users_similar],axis=1)

    
    num = (y.values*x.values).sum(axis=1)
    den = x_length*y_length

    similarity = num/den
    similarity = np.nan_to_num(similarity)
    
    d = list(zip(list(users_similar),similarity))
    new_similarity = sorted(d, key=lambda tup: tup[1], reverse=True)
    
    
    return new_similarity

def ItemUsersRecommendation(new_similarity, new_userDict, k):
    
    '''
    
    input:  new_similarity(List of tuples), new_userDict(Dict), score(int), k(int)
    
    action: recommend items using the ratings of similar users
    
    output: recommendation (Dict), books (Dict)
    
    '''
    
    if new_similarity == []:
        
        return None
    
    if len(new_similarity) > k:
    
        recommendation = np.mean([u[1] for u in new_similarity[:k]]) 
    
    else:
        
        recommendation = np.mean([u[1] for u in new_similarity]) 
        
    return recommendation


def itemUsersScore(new_similarity,new_isbnDict, book_number, k):
    
    if new_similarity == []:
        
        return None
    
    if len(new_similarity) > k:
        
        score = [float(new_isbnDict[str(book_number)][u[0]]) for u in new_similarity[:k] if u[1] !=0.0]
        
    else:
        
        score = [float(new_isbnDict[str(book_number)][u[0]]) for u in new_similarity if u[1] !=0.0]
    
    return np.mean(score)


def CollaborativeFilteringItemUsersRMSE(utility_DataFrame, new_userDict, new_isbnDict, user_number, score_min, book_number, k):
    
    users_rated_book = usersHaveRatedBook(new_isbnDict, new_userDict, book_number, score_min)
    
    if users_rated_book == []:
        
        return None
    
    new_similarity = SimilarityUsers(utility_DataFrame, user_number, users_rated_book)
    
    
    if new_similarity == []:

        return None
    
    score = itemUsersScore(new_similarity,new_isbnDict, book_number, k)
    
    return score
    

In [6]:
def convert(user_number,book_number, index_user_cluster, index_book_cluster):
    
    user_cluster = index_user_cluster[user_number]
    book_cluster = index_book_cluster[book_number]
    
    return user_cluster, book_cluster

In [7]:
def createSampleDict(new_isbnDict, new_userDict, t1, t2):
    

    small_isbnDict = {}

    for book in new_isbnDict:

        temp = new_isbnDict[book]
        i = 0

        for t in temp.values():
            if t != "0":
                i = i+1

        if i >t1:

            small_isbnDict[book] = new_isbnDict[book]   

    small_userDict = {}

    for user in new_userDict:

        temp = new_userDict[user]
        i = 0

        for t in temp.values():
            if t != "0":
                i = i+1

        if i >t2:

            small_userDict[user] = new_userDict[user]
            
            
    return small_isbnDict, small_userDict


def computeMatrices(train_userDict,train_isbnDict,small_userDict,small_isbnDict, dict_row, dict_col):

    n = len(small_isbnDict)
    m = len(small_userDict)
    
    index = sorted(small_userDict.keys())
    columns = sorted(small_isbnDict.keys())

    dict_row = {k:v for v,k in enumerate(index)}
    dict_col = {k:v for v,k in enumerate(columns)}

    u = np.zeros((m,n)) 
    R = np.zeros((m,n))
    for user in train_userDict:
        for isbn in train_userDict[user]:
            try:
                u[dict_row[user]][dict_col[isbn]] = train_userDict[user][isbn]
                R[dict_row[user]][dict_col[isbn]] = 1
            except:
                continue

    for isbn in train_isbnDict:
        for user in train_isbnDict[isbn]:
            try:
                u[dict_row[user]][dict_col[isbn]] = train_isbnDict[isbn][user]
                R[dict_row[user]][dict_col[isbn]] = 1
            except:
                continue
                
    small_utility_DataFrame = pd.DataFrame(u, index = index, columns = columns)
    R = pd.DataFrame(R, index = index, columns = columns)

    return u, R, small_utility_DataFrame


def selectSample(i,b,list_user,small_userDict, small_isbnDict):

    test_index = [k for k in range((i*b),(i+1)*b)]
    train_index = [k for k in range(len(list_user)) if k not in test_index]

    train = list_user[train_index]
    test  = list_user[test_index]

    train_userDict = {}
    train_isbnDict = {isbn:{} for isbn in small_isbnDict}

    for user in train:

        train_userDict[user] = small_userDict[user]

    for isbn in small_isbnDict:
        for user in small_isbnDict[isbn]:

            try:
                train_userDict[user]
                train_isbnDict[isbn][user] = small_isbnDict[isbn][user]
            except:
                continue
                
                
    test_userDict = {}
    test_isbnDict = {isbn:{} for isbn in small_isbnDict}

    for user in test:

        test_userDict[user] = small_userDict[user]

    for isbn in small_isbnDict:
        for user in small_isbnDict[isbn]:

            try:
                test_userDict[user]
                test_isbnDict[isbn][user] = small_isbnDict[isbn][user]
            except:
                continue
                
    return train_userDict, train_isbnDict, test_userDict, test_isbnDict, test


#indices = [i for i in range(len(list_user))]
#indices = np.array(indices)
#k = 5
#b = len(small_userDict)//k


def selectSampleRandom(indices,b,list_user,small_userDict, small_isbnDict):
    
    test_index = np.random.choice(indices, size = b, replace=False)
    
    indices = np.delete(indices,test_index)
    
    train_index = [k for k in range(len(list_user)) if k not in test_index]
    

    train = list_user[train_index]
    test  = list_user[test_index]

    train_userDict = {}
    train_isbnDict = {isbn:{} for isbn in small_isbnDict}

    for user in train:

        train_userDict[user] = small_userDict[user]

    for isbn in small_isbnDict:
        for user in small_isbnDict[isbn]:

            try:
                train_userDict[user]
                train_isbnDict[isbn][user] = small_isbnDict[isbn][user]
            except:
                continue
                
    return train_userDict, train_isbnDict, test_userDict, test_isbnDict, test, indices

In [8]:
def mainItemUsersRMSE_clustering(utility_DataFrame,utility_DataFrame_clusters,R_clusters, new_userDict, new_isbnDict,
                                 index_user_cluster, index_book_cluster, score_min=0, k=3, zeros= "n"):
    

    rmse_dict = {j:{} for j in new_userDict.keys()}

    for user in new_userDict:

        for book in new_userDict[user]:

            if new_userDict[user][book] != "0":
                try:
                    new_isbnDict[book]
                    rmse_dict[user][book] = new_userDict[user][book]
                except:
                    continue
                    
            if new_userDict[user][book] == "0" and zeros != "n":  #if this is true score_min = -1
                try:
                    new_isbnDict[book]
                    rmse_dict[user][book] = "1"
                except:
                    continue

    d = rmse_dict.keys()
    v = []

    for key in d:

        if rmse_dict[key] == {}:

            v.append(key)

    for i in v:

        del rmse_dict[i]

    rmse_vector_itemUsers = []
    
    i = 0
    
    for user in rmse_dict:
        
        i = i+1
        
        if i%1000 == 0:
            
            print("Pause")
            print(np.sqrt((np.array([u[2]-u[3] for u in rmse_vector_itemUsers])**2).sum()/len(rmse_vector_itemUsers)))
            time.sleep(30)

        for isbn in rmse_dict[user]:
            
            prediction_score = None
            user_number = user
            book_number = isbn
            
            try:
                
                prediction_score = CollaborativeFilteringItemUsersRMSE(utility_DataFrame, new_userDict, new_isbnDict, 
                                                                   user_number, score_min, book_number, k)
                true_score = utility_DataFrame[book_number][user_number]
            
            except:
                continue

            if prediction_score == None:
                
                user_cluster, book_cluster = convert(user_number,book_number, index_user_cluster, index_book_cluster)
                
                if R_clusters[book_cluster][user_cluster] == 1:
                
                    prediction_score = utility_DataFrame_clusters[book_cluster][user_cluster]
                    true_score = utility_DataFrame[book_number][user_number]

            #print(true_score,prediction_score)

            if prediction_score != None:

                rmse_vector_itemUsers.append(tuple([user_number, book_number, prediction_score, true_score]))


    rmse = np.sqrt((np.array([u[2]-u[3] for u in rmse_vector_itemUsers])**2).sum()/len(rmse_vector_itemUsers))
    
    return rmse, rmse_vector_itemUsers

In [9]:
small_isbnDict, small_userDict = createSampleDict(new_isbnDict, new_userDict, 20, 20)

In [10]:
u_clusters, R_clusters, utility_DataFrame_clusters = computeMatrices(CLUSTERS_USERS,CLUSTERS_ITEMS,
                                                                           CLUSTERS_USERS,CLUSTERS_ITEMS,
                                                                            clusters_dict_row,clusters_dict_col)

u, R, utility_DataFrame = computeMatrices(small_userDict,small_isbnDict,new_userDict,new_isbnDict,dict_row,dict_col)

In [14]:
rmse_test, rmse_vector_itemUsers = mainItemUsersRMSE_clustering(utility_DataFrame,utility_DataFrame_clusters,R_clusters,
                                                                small_userDict,small_isbnDict,
                                                                index_user_cluster,index_book_cluster,
                                                                score_min=0,k=3)

In [13]:
rmse_vector_itemUsers[:100]

[('2766', '0440219078', 8.0, 8.0),
 ('2766', '0440204194', 9.0, 8.0),
 ('2766', '0684801221', 10.0, 10.0),
 ('2766', '0440211263', 8.6666666666666661, 8.0),
 ('2766', '0449005615', 9.3333333333333339, 9.0),
 ('2766', '0767902521', 8.3333333333333339, 8.0),
 ('2766', '0440221595', 8.6666666666666661, 8.0),
 ('2766', '0440237688', 9.0, 7.0),
 ('2766', '0679751602', 7.333333333333333, 6.0),
 ('2766', '0060958022', 7.666666666666667, 8.0),
 ('2766', '0971880107', 5.666666666666667, 6.0),
 ('2766', '0553279912', 7.333333333333333, 7.0),
 ('2766', '0446611867', 8.6666666666666661, 7.0),
 ('2766', '0316666343', 8.3333333333333339, 9.0),
 ('2766', '014034294X', 8.3333333333333339, 9.0),
 ('2766', '0425178102', 7.0, 5.0),
 ('2766', '0446310786', 10.0, 10.0),
 ('2766', '1558745718', 8.3333333333333339, 7.0),
 ('2766', '044048474X', 7.666666666666667, 7.0),
 ('2766', '0385508417', 9.0, 7.0),
 ('2766', '0446516538', 6.0, 6.0),
 ('2766', '0670030643', 7.666666666666667, 8.0),
 ('162001', '014026568

# K-fold cross validation for collaborative filtering user-based

In [11]:
def itemUsers_KfoldsCV_clustering(small_userDict,small_isbnDict, dict_row, dict_col,CLUSTERS_USERS,CLUSTERS_ITEMS,
                                                                            clusters_dict_row,clusters_dict_col, kk):

    RMSE = []
    
    list_user = np.array(sorted(small_userDict.keys()))
    b = len(small_userDict)//kk
    
    u_clusters, R_clusters, utility_DataFrame_clusters = computeMatrices(CLUSTERS_USERS,CLUSTERS_ITEMS,
                                                                           CLUSTERS_USERS,CLUSTERS_ITEMS,
                                                                            clusters_dict_row,clusters_dict_col)

    for iterator in range(kk):

        train_userDict, train_isbnDict, test_userDict, test_isbnDict, test =  selectSample(iterator,b,list_user,
                                                                                           small_userDict, small_isbnDict)

        u, R, small_utility_DataFrame = computeMatrices(train_userDict,small_isbnDict,small_userDict,small_isbnDict,dict_row,dict_col)

        rmse_test, rmse_vector_itemUsers = mainItemUsersRMSE_clustering(small_utility_DataFrame,utility_DataFrame_clusters,R_clusters, 
                                                             test_userDict, test_isbnDict,index_user_cluster,index_book_cluster,
                                                             score_min=0, k=3)
        print("Ok")

        RMSE.append(rmse_test)
        
    return RMSE

In [9]:
small_userDict = {k:new_userDict[k] for k in list(new_userDict.keys())[:5000]}
small_isbnDict = {k:new_isbnDict[k] for k in list(new_isbnDict.keys())[:5000] }

In [12]:
RMSE = itemUsers_KfoldsCV_clustering(new_userDict,new_isbnDict, dict_row, dict_col,CLUSTERS_USERS,CLUSTERS_ITEMS,
                                                                            clusters_dict_row,clusters_dict_col, 5)

Pause
1.24302093629
Pause
1.2230951783
Pause
1.22064005137
Ok
Pause
1.18027283173
Pause
1.17665939392
Pause
1.16800211558
Ok
Pause
1.23725059508
Pause
1.1997828053
Pause
1.20466398197
Ok
Pause
1.18887286193
Pause
1.20751138263
Pause
1.19235177416
Ok
Pause
1.19091298041
Pause
1.19009579166
Pause
1.17096646174
Ok


In [13]:
print("rmse collaborative filtering user-based with clustering: ",np.mean(RMSE))

rmse collaborative filtering user-based with clustering:  1.19090725471
